# TigerGraph Graph Data Science Library 101 - Path Finding Algorithm

This notebook shows the examples of using the most common path finding algorithms in TigerGraph Graph Science Library. More detailed explanations of these algorithms can be four in the official documentation (https://docs.tigergraph.com/graph-ml/current/pathfinding-algorithms/).  

## Step 1: Setting things up
- Connect and Load data

In [1]:
from pyTigerGraph.datasets import Datasets

dataset = Datasets("ldbc_snb")

Downloading:   0%|          | 0/286678171 [00:00<?, ?it/s]

In [2]:
from pyTigerGraph import TigerGraphConnection
import json

# Read in DB configs
with open('../config.json', "r") as config_file:
    config = json.load(config_file)

conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"],
)

In [3]:
conn.ingestDataset(dataset, getToken=config["getToken"])

---- Checking database ----
A graph with name ldbc_snb already exists in the database. Please drop it first before ingesting.


- Visualize the graph schema 

In [4]:
from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

- Get basic stats, e.g., counts of nodes & edges

In [5]:
vertices = conn.getVertexTypes()
for vertex in vertices:
    print("Node count: ({} : {}) ".format(vertex, conn.getVertexCount(vertex)))

Node count: (Comment : 2052169) 
Node count: (Post : 1003605) 
Node count: (Company : 1575) 
Node count: (University : 6380) 
Node count: (City : 1343) 
Node count: (Country : 111) 
Node count: (Continent : 6) 
Node count: (Forum : 90492) 
Node count: (Person : 9892) 
Node count: (Tag : 16080) 
Node count: (Tag_Class : 71) 


In [6]:
import pprint
print("Edges counts: ")
pprint.pprint(conn.getEdgeCount())

Edges counts: 
{'Container_Of': 1003605,
 'Container_Of_Reverse': 1003605,
 'Has_Creator': 3055774,
 'Has_Creator_Reverse': 3055774,
 'Has_Interest': 229166,
 'Has_Interest_Reverse': 229166,
 'Has_Member': 1611869,
 'Has_Member_Reverse': 1611869,
 'Has_Moderator': 90492,
 'Has_Moderator_Reverse': 90492,
 'Has_Tag': 3721417,
 'Has_Tag_Reverse': 3721417,
 'Has_Type': 16080,
 'Has_Type_Reverse': 16080,
 'Is_Located_In': 3073621,
 'Is_Located_In_Reverse': 3073621,
 'Is_Part_Of': 1454,
 'Is_Part_Of_Reverse': 1454,
 'Is_Subclass_Of': 70,
 'Is_Subclass_Of_Reverse': 70,
 'Knows': 180623,
 'Likes': 2190095,
 'Likes_Reverse': 2190095,
 'Reply_Of': 2052169,
 'Reply_Of_Reverse': 2052169,
 'Study_At': 7949,
 'Study_At_Reverse': 7949,
 'Work_At': 21654,
 'Work_At_Reverse': 21654}


## Step 2: Leveraging pyTigerGraph’s featurizer to run Path Finding algorithms


In [7]:
feat = conn.gds.featurizer()

In [8]:
feat.listAlgorithms("Path")

Available algorithms for Path:
  bfs:
    01. name: tg_bfs
  cycle_detection:
    02. name: tg_cycle_detection_count
  shortest_path:
    03. name: tg_shortest_ss_no_wt
Call runAlgorithm() with the algorithm name to execute it


## tg_bfs
Breadth-First Search Algorithm from a single source node

In [9]:
params = {
    "v_type_set": ["Person"],
    "e_type_set": ["Knows"],
    "max_hops": 2,
    "v_start": {"id": "21990232556463", "type": "Person"}, ##{"id": "vertex_id", "type": "vertex_type"}
    "print_results": True,
    "result_attribute": "",
    "file_path": "",
    "display_edges": False
  }

In [10]:
res = feat.runAlgorithm("tg_bfs", params=params)

In [11]:
len(res[0]['Start']), res[0]['Start'][:10]

(4069,
 [{'v_id': '21990232566155',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '8796093029876',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '21990232565733',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '6597069769055',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '2199023263448',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '8796093027437',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '2199023257517',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '6597069770520',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '17592186052664',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}},
  {'v_id': '19791209302191',
   'v_type': 'Person',
   'attributes': {'Start.@sum_step': 2}}])

## tg_shortest_path
Single-source shortest path algorithm, with unweighted edges.

In [12]:
params = {
    "source": {"id": "21990232556463", "type": "Person"}, ##{"id": "vertex_id", "type": "vertex_type"}
    "v_type_set": ["Person"],
    "e_type_set": ["Knows"],
    "print_limit": 20,
    "print_results": True,
    "result_attribute": "",
    "file_path": "",
    "display_edges": False
}

In [13]:
res = feat.runAlgorithm("tg_shortest_ss_no_wt", params=params)

In [14]:
print(len(res[0]['ResultSet']))
res[0]['ResultSet'][:5]

20


[{'v_id': '15393162794623',
  'v_type': 'Person',
  'attributes': {'ResultSet.@min_dis': 3,
   'ResultSet.@path_list': ['21990232556463',
    '10995116278291',
    '19791209304170',
    '15393162794623']}},
 {'v_id': '21990232566155',
  'v_type': 'Person',
  'attributes': {'ResultSet.@min_dis': 2,
   'ResultSet.@path_list': ['21990232556463', '2783', '21990232566155']}},
 {'v_id': '8796093029876',
  'v_type': 'Person',
  'attributes': {'ResultSet.@min_dis': 2,
   'ResultSet.@path_list': ['21990232556463',
    '6597069777240',
    '8796093029876']}},
 {'v_id': '35184372098404',
  'v_type': 'Person',
  'attributes': {'ResultSet.@min_dis': 3,
   'ResultSet.@path_list': ['21990232556463',
    '32985348834375',
    '17592186053137',
    '35184372098404']}},
 {'v_id': '21990232565733',
  'v_type': 'Person',
  'attributes': {'ResultSet.@min_dis': 2,
   'ResultSet.@path_list': ['21990232556463',
    '32985348834375',
    '21990232565733']}}]

## tg_cycle_detection_count
This is a distributed algorithm for detecting all the cycles on large-scale directed graphs.

In [15]:
params =  {
    "v_type_set": ["Person"],
    "e_type_set": ["Knows"],
    "depth": 2,
    "batches": 2,
    "print_results": True
}

In [16]:
res = feat.runAlgorithm("tg_cycle_detection_count", params=params)

In [17]:
# Display Results
res

[{'cycles': 180623}]